# prologue

### set up notebook and load package

In [1]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.datasets_proprietary as dsp
import CHIRPS.reproducible as rp
import multiprocessing as mp

## Sensitivity Analysis

In [2]:
# Optional Memory and Computation Cost Management
# CHIRPS is time economical but memory intensive to compute for lots of instances at once.
forest_walk_async=True
explanation_async=False
n_cores = mp.cpu_count()-2

# How many instances from the test set do you want to explain?
# A number bigger than the test set will be interpreted as 'all'
n_instances = 100
start_instance = 0 # here can opt to start at a specific instance, diagnostic if something crashes

# model = 'RandomForest'
# model = 'AdaBoost1'
# model = 'AdaBoost2'
model = 'GBM'

# prepare data
datasets = [
#         ds.adult,
        ds.bankmark,
#         ds.car,
#         ds.cardio,
#         ds.credit,
#         ds.german,
#         ds.lending_tiny_samp,
#         ds.nursery,
#         ds.rcdv,
       ]

# location to save results
project_dir = '/datadisk/whiteboxing/2020/GBM'
# project_dir = 'V:\\whiteboxing\\2020\\GBM' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\2020\\GBM'

random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

# CHIRPS default parameters - see papers for details
precis_threshold = 0.99
merging_bootstraps = 20 # how many training bootstraps to test improvement in growing rule?
pruning_bootstraps = 20 # how many training bootstraps to test deterioration in pruning rule?
delta = 0.2 # pruning deterioration tolerance paramater

tuning = {'override' : False}

if model == 'RandomForest':
    tuning.update({'grid' : {'n_estimators': [(i + 1) * 200 for i in range(8)], 'max_depth' : [32]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)

    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.3, 0.2], 18), 2)
    weighting = np.repeat(['kldiv', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'disc_path_eqcounts' : False,
                        'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                        'precis_threshold' : precis_threshold,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : 'majority',
                        'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}
    
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'explanation_async' : explanation_async,
                    'n_cores' : n_cores,
                    'save_sensitivity_path' : 'rf_sensitivity'}
        
        print('Sensitivity round: ' + str(kwargs))
        rp.do_benchmarking(bi_copy, verbose, **control)
        
elif model in ('AdaBoost1', 'AdaBoost2'):
    if model == 'AdaBoost1':
        algo = 'SAMME'
        save_sensitivity_path = 'ada1_sensitivity'
        support_paths = np.tile(np.tile([0.05, 0.02, 0.01], 16), 2)
        n_kwargs = 36
    else:
        algo = 'SAMME.R'
        save_sensitivity_path = 'ada2_sensitivity'
        support_paths = np.tile(np.tile([0.005, 0.002, 0.001], 16), 2)
        n_kwargs = 72
        
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)
    
    disc_path_bins = np.tile(np.tile(np.repeat([4, 8], 3), 6), 2)
    disc_path_eqcounts = np.tile(np.tile(np.repeat([True, False], 6), 6), 2)
    weighting = np.tile(np.repeat(['chisq', 'kldiv', 'nothing'], 12), 2)
    which_trees = np.repeat(['majority', 'conf_weighted'], 36)

    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                        'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                        'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : wchtr,
                        'delta' : delta} 
    for k, dpb, dpeq, w, sp, wchtr \
                   in zip(range(n_kwargs), disc_path_bins, disc_path_eqcounts, weighting, support_paths, which_trees)}

    for kwargs in kwargs_grid: # range(39, 72): # 
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'explanation_async' : explanation_async,
                    'n_cores' : n_cores,
                    'save_sensitivity_path' : save_sensitivity_path}
        
        print('Sensitivity round: ' + str(kwargs))
        rp.do_benchmarking(bi_copy, verbose, **control)

else: # GBM
    tuning.update({'grid' : {'subsample' : [(i + 1) * 0.25 for i in range(3)],
                    'n_estimators': [i * 200 for i in range(1, 9)],
                    'max_depth' : [i for i in range(1, 5)],
                    'learning_rate': np.full(4, 10.0)**[i for i in range(-3, 1)]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)

    support_paths = np.tile([0.05, 0.1], 4)
    disc_path_bins = np.tile(np.repeat([4, 8], 2), 2)
    disc_path_eqcounts = np.repeat([True, False], 4)

    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                    'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                    'score_func' : 1, 'weighting' : 'kldiv',
                    'support_paths' : sp,
                    'merging_bootstraps' : merging_bootstraps,
                    'pruning_bootstraps' : pruning_bootstraps,
                    'which_trees' : 'targetclass',
                    'delta' : 0.2} 
        for k, dpb, dpeq, sp in zip(range(16), disc_path_bins, disc_path_eqcounts, support_paths)}
    
    for kwargs in kwargs_grid: # range(39, 72): # 
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'explanation_async' : explanation_async,
                    'n_cores' : n_cores,
                    'save_sensitivity_path' : 'gbm_sensitivity'}

        print('Sensitivity round: ' + str(kwargs))
        rp.do_benchmarking(bi_copy, verbose, **control)
    
print('finished')

Preprocessing bankmark data and model for bankmark with random state = 123
Split data into main train-test and build forest


/home/julianhatwell/anaconda3/envs/B3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.9202
Best parameters:{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 800, 'subsample': 0.5, 'random_state': 123}


Sensitivity round: 0
Beginning benchmark for bankmark data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 4, 'disc_path_eqcounts': True, 'score_func': 1, 'weighting': 'kldiv', 'support_paths': 0.05, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'targetclass', 'delta': 0.2}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 100 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 2.8694 seconds

Running GBHIPS on a batch of 100 instances... (please wait)
target class
0
Working on GBHIPS for instance 0 of 100
800
100
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.05


/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1535: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1540: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans


found 6 patterns from 78 trees for batch_idx 0
start score sort for batch_idx 0 (6) patterns
start merge rule for batch_idx 0 (6) patterns
[('nr.employed', False, 5088.0)]
0.9512125125556034 0.4850689833487866 0.410501991059384 0.5326522081121445
merge complete for batch_idx 0 (6) patterns
start get explainer for batch_idx 0
target class
0
800
100
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.05


...

/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1535: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1540: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans


found 2 patterns from 84 trees for batch_idx 53
start score sort for batch_idx 53 (2) patterns
start merge rule for batch_idx 53 (2) patterns
[('duration', True, 448.5)]
0.9456120943952803 0.4394741942816801 0.4855492639692778 0.943820719774967
merge complete for batch_idx 53 (2) patterns
start get explainer for batch_idx 53
target class
1
800
100
as_chirps for batch_idx 54
start mining for batch_idx 54 with support = 0.05
found 2 patterns from 17 trees for batch_idx 54
start score sort for batch_idx 54 (2) patterns
start merge rule for batch_idx 54 (2) patterns


/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1535: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(uppers, upper_bins)[0]).round(5) # can result in nans if no value falls into bin
/home/julianhatwell/Documents/github/explain_te/CHIRPS/structures.py:1540: RuntimeWarning: invalid value encountered in true_divide
  np.histogram(lowers, lower_bins)[0]).round(5) # can result in nans


[('euribor3m', True, 1.29492)]
0.2969376302709636 0.1665386710115848 0.9595829081073014 0.9223059824609899
merge complete for batch_idx 54 (2) patterns
start get explainer for batch_idx 54
target class
0
Working on GBHIPS for instance 55 of 100
800
100
[{'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [24.0, 24.0, 24.0, 24.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001079904105317782, 'forest_pred_lodds': -6.8298025455387315, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.764682741472379}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [157.0, 157.0, 157.0, 157.0], 'thres

ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
a = [{'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [24.0, 24.0, 24.0, 24.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001079904105317782, 'forest_pred_lodds': -6.8298025455387315, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.764682741472379}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [157.0, 157.0, 157.0, 157.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0018458508699013198, 'forest_pred_lodds': -6.292967375148486, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.227847571082133}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [622.0, 622.0, 622.0, 622.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.19114957591660098, 'forest_pred_lodds': -1.4425577685293234, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 0.6225620355370294}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [608.0, 608.0, 608.0, 608.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.026337836460229438, 'forest_pred_lodds': -3.6100578347944823, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.5449380307281295}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [66.0, 66.0, 66.0, 66.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0010056377671267742, 'forest_pred_lodds': -6.901127200824785, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.836007396758433}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [700.0, 700.0, 700.0, 700.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.842650239842132, 'forest_pred_lodds': 1.6780808733000099, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 3.743200677366363}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [264.0, 264.0, 264.0, 264.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.00355059019865405, 'forest_pred_lodds': -5.6370845276670725, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.5719647236007197}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [504.0, 504.0, 504.0, 504.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.12559058757461877, 'forest_pred_lodds': -1.940521389709847, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 0.12459841435650576}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [214.0, 214.0, 214.0, 214.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0018285651271910662, 'forest_pred_lodds': -6.302393464191972, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.23727366012562}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [414.0, 414.0, 414.0, 414.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.027035193983695267, 'forest_pred_lodds': -3.5832084128539754, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.5180886087876226}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [60.0, 60.0, 60.0, 60.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0009766668747490357, 'forest_pred_lodds': -6.930387787443064, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.865267983376711}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [187.0, 187.0, 187.0, 187.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.006661581307420681, 'forest_pred_lodds': -5.004714520391347, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.939594716324994}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [20.0, 20.0, 20.0, 20.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.00086263828295437, 'forest_pred_lodds': -7.0546520832774195, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.989532279211067}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [709.0, 709.0, 709.0, 709.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.4259453408806592, 'forest_pred_lodds': -0.2984135864872039, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 1.7667062175791488}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [54.0, 54.0, 54.0, 54.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0009296700485538134, 'forest_pred_lodds': -6.979750718810095, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.914630914743743}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [114.0, 114.0, 114.0, 114.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.002212674818518174, 'forest_pred_lodds': -6.111338043783638, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.046218239717286}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [65.0, 65.0, 65.0, 65.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.004179458163019587, 'forest_pred_lodds': -5.47338545040929, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.408265646342937}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [212.0, 212.0, 212.0, 212.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001704331186516051, 'forest_pred_lodds': -6.372876725964829, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.307756921898477}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [90.0, 90.0, 90.0, 90.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0016692035686438258, 'forest_pred_lodds': -6.393738073071399, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.328618269005046}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [87.0, 87.0, 87.0, 87.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0017019452490261153, 'forest_pred_lodds': -6.374280022754942, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.309160218688589}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [48.0, 48.0, 48.0, 48.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001298514817586956, 'forest_pred_lodds': -6.64523475699738, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.5801149529310266}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [299.0, 299.0, 299.0, 299.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.00460111424776572, 'forest_pred_lodds': -5.37684504508904, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.3117252410226876}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [88.0, 88.0, 88.0, 88.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0010906187794148107, 'forest_pred_lodds': -6.819918842341332, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.75479903827498}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [323.0, 323.0, 323.0, 323.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.005647551494400944, 'forest_pred_lodds': -5.1708696323644725, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.1057498282981197}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [310.0, 310.0, 310.0, 310.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.004151464480590924, 'forest_pred_lodds': -5.480134014366632, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.415014210300279}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [250.0, 250.0, 250.0, 250.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0023202402646272657, 'forest_pred_lodds': -6.063761600133777, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.998641796067424}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [62.0, 62.0, 62.0, 62.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0008989002676873587, 'forest_pred_lodds': -7.013439162069789, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.948319358003436}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [47.0, 47.0, 47.0, 47.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0012518061293150921, 'forest_pred_lodds': -6.681915276791314, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.616795472724961}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [144.0, 144.0, 144.0, 144.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.021300540389306327, 'forest_pred_lodds': -3.827492165497993, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.7623723614316402}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [278.0, 278.0, 278.0, 278.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.005464183164296896, 'forest_pred_lodds': -5.204061468840348, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.1389416647739954}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [445.0, 445.0, 445.0, 445.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.019975497060168792, 'forest_pred_lodds': -3.89307119889556, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.8279513948292072}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [154.0, 154.0, 154.0, 154.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0056366455579216395, 'forest_pred_lodds': -5.172813558251948, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.1076937541855956}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [163.0, 163.0, 163.0, 163.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0014770291144424106, 'forest_pred_lodds': -6.5162444427528765, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.451124638686524}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [153.0, 153.0, 153.0, 153.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0018570735375395327, 'forest_pred_lodds': -6.286894597176373, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.22177479311002}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [301.0, 301.0, 301.0, 301.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.005430659709781504, 'forest_pred_lodds': -5.210249199554203, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.14512939548785}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [227.0, 227.0, 227.0, 227.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.531050501630677, 'forest_pred_lodds': 0.12436204010336573, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 2.1894818441697184}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [402.0, 402.0, 402.0, 402.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.025656465291850675, 'forest_pred_lodds': -3.6369683484786255, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.5718485444122727}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [153.0, 153.0, 153.0, 153.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0017891197122221704, 'forest_pred_lodds': -6.3242408388764, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.259121034810047}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [185.0, 185.0, 185.0, 185.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0016239694361796125, 'forest_pred_lodds': -6.421256568002292, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.35613676393594}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [172.0, 172.0, 172.0, 172.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001805419468491629, 'forest_pred_lodds': -6.315155270477621, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.250035466411267}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [253.0, 253.0, 253.0, 253.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0029862151042491783, 'forest_pred_lodds': -5.810757862126893, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.7456380580605404}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [172.0, 172.0, 172.0, 172.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001540232786064712, 'forest_pred_lodds': -6.474280294033843, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.4091604899674905}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [191.0, 191.0, 191.0, 191.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0014293062066016557, 'forest_pred_lodds': -6.549135794013021, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.484015989946668}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [191.0, 191.0, 191.0, 191.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0014617488090884864, 'forest_pred_lodds': -6.526658927418968, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.461539123352615}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [706.0, 706.0, 706.0, 706.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.7700856325502085, 'forest_pred_lodds': 1.2087947956074936, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 3.273914599673846}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [121.0, 121.0, 121.0, 121.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001837877325544519, 'forest_pred_lodds': -6.2973044324287715, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.232184628362418}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [241.0, 241.0, 241.0, 241.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.002892971318198265, 'forest_pred_lodds': -5.842574003196618, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.7774541991302653}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [197.0, 197.0, 197.0, 197.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.21414465505057315, 'forest_pred_lodds': -1.3001209911420433, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 0.7649988129243095}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [64.0, 64.0, 64.0, 64.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.004661662693383012, 'forest_pred_lodds': -5.363710531198722, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.298590727132369}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [9.0, 9.0, 9.0, 9.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0008498717744830653, 'forest_pred_lodds': -7.069574840288542, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -5.00445503622219}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [253.0, 253.0, 253.0, 253.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.002557163542076803, 'forest_pred_lodds': -5.966296187473827, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.9011763834074746}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [1037.0, 1037.0, 1037.0, 1037.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.6511554420698727, 'forest_pred_lodds': 0.6241219577721396, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 2.6892417618384923}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [61.0, 61.0, 61.0, 61.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.003302082137312427, 'forest_pred_lodds': -5.7098945127326495, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.6447747086662967}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [135.0, 135.0, 135.0, 135.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.001609526638942796, 'forest_pred_lodds': -6.4302043329766025, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.36508452891025}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [458.0, 458.0, 458.0, 458.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.9496773432673947, 'forest_pred_lodds': 2.937666880487039, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 5.002786684553392}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [178.0, 178.0, 178.0, 178.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.01215157712692199, 'forest_pred_lodds': -4.398070301785847, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.332950497719494}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [43.0, 43.0, 43.0, 43.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.009737989530255127, 'forest_pred_lodds': -4.6219348825365945, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.5568150784702417}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [154.0, 154.0, 154.0, 154.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0015641780965941216, 'forest_pred_lodds': -6.458829368139045, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.393709564072692}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [383.0, 383.0, 383.0, 383.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.07403328431706394, 'forest_pred_lodds': -2.5263235096650076, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -0.46120370559865487}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [9.0, 9.0, 9.0, 9.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.003111819077613044, 'forest_pred_lodds': -5.769431140515369, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.7043113364490163}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [11.0, 11.0, 11.0, 11.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0011849937597495197, 'forest_pred_lodds': -6.73683207402306, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.671712269956707}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [518.0, 518.0, 518.0, 518.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.07486015709645769, 'forest_pred_lodds': -2.514323106592209, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -0.44920330252585616}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [135.0, 135.0, 135.0, 135.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0054001551397895465, 'forest_pred_lodds': -5.2159128075503345, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.1507930034839817}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [90.0, 90.0, 90.0, 90.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0012550632626223304, 'forest_pred_lodds': -6.6793134476907206, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.614193643624368}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [745.0, 745.0, 745.0, 745.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.7369585917521101, 'forest_pred_lodds': 1.0302202401487306, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 3.0953400442150834}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [201.0, 201.0, 201.0, 201.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0015568081116570291, 'forest_pred_lodds': -6.463559614881225, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.398439810814873}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [185.0, 185.0, 185.0, 185.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0018180447048619128, 'forest_pred_lodds': -6.308173994042193, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.24305418997584}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [81.0, 81.0, 81.0, 81.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0016538731599767007, 'forest_pred_lodds': -6.402980129837132, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.337860325770778}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [91.0, 91.0, 91.0, 91.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.011722412787138951, 'forest_pred_lodds': -4.434460984833312, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.3693411807669595}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [68.0, 68.0, 68.0, 68.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0018528717674440123, 'forest_pred_lodds': -6.289163946290431, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.224044142224079}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [350.0, 350.0, 350.0, 350.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.1388541850433406, 'forest_pred_lodds': -1.8248394913890438, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 0.24028031267730898}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [276.0, 276.0, 276.0, 276.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.004242460957132336, 'forest_pred_lodds': -5.458360277967077, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.393240473900724}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [213.0, 213.0, 213.0, 213.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0022260806497060243, 'forest_pred_lodds': -6.105284233959258, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.040164429892906}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [247.0, 247.0, 247.0, 247.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.015572895751447288, 'forest_pred_lodds': -4.146527900608131, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.0814080965417783}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [446.0, 446.0, 446.0, 446.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.036497640373407386, 'forest_pred_lodds': -3.273327325672275, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.2082075216059223}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [186.0, 186.0, 186.0, 186.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0017260929601852328, 'forest_pred_lodds': -6.360167244656135, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.295047440589782}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [110.0, 110.0, 110.0, 110.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0012952509070157416, 'forest_pred_lodds': -6.647754761537039, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.582634957470686}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [355.0, 355.0, 355.0, 355.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.962230318582451, 'forest_pred_lodds': 3.2377471365287107, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 5.302866940595063}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [173.0, 173.0, 173.0, 173.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.16361580123050057, 'forest_pred_lodds': -1.6315670713786417, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 0.4335527326877111}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [22.0, 22.0, 22.0, 22.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0018863064699614874, 'forest_pred_lodds': -6.27124652285723, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.206126718790877}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [323.0, 323.0, 323.0, 323.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.05216204954970521, 'forest_pred_lodds': -2.8998283389391126, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -0.8347085348727599}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [149.0, 149.0, 149.0, 149.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0017919272247194863, 'forest_pred_lodds': -6.322670041812848, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.257550237746495}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [180.0, 180.0, 180.0, 180.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0014136073756862108, 'forest_pred_lodds': -6.560195811849771, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.4950760077834175}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [113.0, 113.0, 113.0, 113.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.012795480173821715, 'forest_pred_lodds': -4.34578523442385, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.2806654303574976}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [81.0, 81.0, 81.0, 81.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0010063348242946633, 'forest_pred_lodds': -6.900433593829491, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.835313789763138}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [128.0, 128.0, 128.0, 128.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0012828563730578658, 'forest_pred_lodds': -6.65738246585206, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.592262661785707}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [746.0, 746.0, 746.0, 746.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 1, 'forest_pred_probas': 0.8383826052867845, 'forest_pred_lodds': 1.646242784994311, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 3.7113625890606636}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [361.0, 361.0, 361.0, 361.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.008746969833851443, 'forest_pred_lodds': -4.730262494169023, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.66514269010267}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [246.0, 246.0, 246.0, 246.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.006717216639080416, 'forest_pred_lodds': -4.996341522181066, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.931221718114713}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [267.0, 267.0, 267.0, 267.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0061351997902250105, 'forest_pred_lodds': -5.087558538243562, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.0224387341772094}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [337.0, 337.0, 337.0, 337.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.005588226551327094, 'forest_pred_lodds': -5.181489396833953, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.1163695927676005}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [295.0, 295.0, 295.0, 295.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.007798994258454211, 'forest_pred_lodds': -4.845930929356342, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.7808111252899894}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [322.0, 322.0, 322.0, 322.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.012952677334313625, 'forest_pred_lodds': -4.333415473507918, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.268295669441565}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [356.0, 356.0, 356.0, 356.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.010008461518128951, 'forest_pred_lodds': -4.594265509081793, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -2.52914570501544}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [31.0, 31.0, 31.0, 31.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0011589871503479962, 'forest_pred_lodds': -6.759049142234472, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.69392933816812}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [652.0, 652.0, 652.0, 652.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': True, 'forest_pred_class': 0, 'forest_pred_probas': 0.2358699191854922, 'forest_pred_lodds': -1.1754575743470246, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': 0.8896622297193282}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [44.0, 44.0, 44.0, 44.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.003747317228024128, 'forest_pred_lodds': -5.582960744873839, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.517840940807486}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [208.0, 208.0, 208.0, 208.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0017966714957745199, 'forest_pred_lodds': -6.320021207471971, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -4.2549014034056185}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [493.0, 493.0, 493.0, 493.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0360068634296786, 'forest_pred_lodds': -3.287374603735913, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -1.2222547996695603}, {'path': {'feature_idx': [54, 54, 54, 54], 'feature_name': ['duration', 'duration', 'duration', 'duration'], 'feature_value': [226.0, 226.0, 226.0, 226.0], 'threshold': [1737.5, 1616.5, 1364.5, 1300.0], 'leq_threshold': [True, True, True, True]}, 'estimator_weight': 0.000503548028069302, 'pred_class': 1, 'pred_value': 0.000503548028069302, 'agree_sign_delta': False, 'forest_pred_class': 0, 'forest_pred_probas': 0.0031512266238322094, 'forest_pred_lodds': -5.756807295357666, 'prior_probas': 0.11253350149771402, 'prior_lodds': -2.0651198040663528, 'delta_lodds': -3.691687491291313}]

In [ ]:
a

In [ ]:
paths_info, paths_weights, pred_class = [i for i in map(list, zip(*[itemgetter('path', 'estimator_weight', 'pred_class')(a[target_class_slice][batch_idx][pd]) for pd in a])]

In [ ]:
from operator import itemgetter
paths_info, paths_weights, pred_class = [i for i in map(list, zip(*[itemgetter('path', 'estimator_weight', 'pred_class')(a[pd]) for pd in range(len(a))]))]

In [ ]:
len(paths_info)